**there was a small bug on the morning of the 24th if you tried to use it then and it wasn't working try it now :)**

# **First, thanks to all for checking out my notebook!** Hopefully this code will be of use for anyone using YOLOv5 (TF version), and even thoes who just want the labels separated out in this way:  [[R1][R2][R3]] where R1, R2, and R3 represent radiologist 1, 2, and 3's findings and corresponding box coordinates.


This program was designed to use train.csv to create inputs to a YOLOv5 architecture network. It creates and saves 3 useful files: a list of the unique image IDs, a list of the findings in each image searated into a list for each radiologist, and a list of the box centers width and heighth. People should be able to use it as a generic data loader for the labels and boxes. It groups the labels by image to save computation and by radiologist to better train the networks. The latter part seems to be of more debate: How can we use the radiologist ID data to help us in the contest? So far, we believe at least grouping them into the three radiologists who viewed each image should be done, but perhaps there is more? The reason for separating the radiologists is so that we are training a network to output any findings as one single radiologist would as opposed to a team of three. The drawback here is that we have to feed the network training data where each input will have 3 ground truths. Because we don't know which radiologist is accurately labeling the image, we will have to treat all 3 as if they are equally right even when they disagree. This certainly seems like it could be confusing for the network depending on how much the radiologists findings overlap, but it may actually be helpful in the sense of adding a bit of noise to the data. I would love to hear your opinions on this!


**#NOTE#** I try to break the code into separate blocks below, but if you try this, it doesn't populate the lists for some reason... I am not super familiar working in these kaggle environments, it seems like maybe a variable is getting lost somewhere between cells as it does work right in a single cell. Maybe one of you more experienced kaggler's can tell me where it's going wrong...

In [ ]:
import pandas as pd
import numpy as np
import os
import pickle


#Read in the labels from the text file in csv format#
train_df = pd.read_csv("../input/vinbigdata-chest-xray-abnormalities-detection/train.csv")

print(train_df.head())

#Initialize a list for the unique IDs in all the findings#
image_ids = []

#Using the index method to return the number of rows in the label file, in other#
#words it is the total number of findings in all of the images from all Radiologists)#
f = train_df.index

print(len(f))

#To group by image, we first need a list of the unique images names from the findings#
#We can do this easily using the .unique() method on the image_id column of the labels#
#file#
image_ids = list(train_df['image_id'].unique())
#We also need a list of image IDs for each finding#
findings = list(train_df['image_id'])

#Testing that lists are properly initialized#
print(findings[59000])

print(image_ids[12])

#Initializing lists labels_img_rad will be used to create final_labels and boxes_img_rad will do the#
#same for the associated final_boxes#
labels_img_rad = []
boxes_img_rad = []
final_labels = []
final_boxes = []

#The structure for final_labels and final_boxes is a list for each image containing three lists for the#
#findings of the three radiologists. In final_labels each list is number corresponding to the category of#
#that radiologist's findings, while in final_boxes each list is a list of 4 numbers corresponding to#
#the bounding box for that radiologist's findings. To get to this final structure, I will use labels_img_rad#
#and boxes_img_rad to store the needed information.#

#This first loop initializes said lists
for img in image_ids:
   labels_img_rad.append([[],img])
   boxes_img_rad.append([[],img])
   final_labels.append([[],[],[]])
   final_boxes.append([[],[],[]])
i = 0
#Next, add the radiologist ID and their findings (labels and box coordinates) into labels_img_rad and boxes_img_rad#
for name in findings:
   idx = image_ids.index(name)
   split = list(train_df.iloc[i])
   labels_img_rad[idx][0].append({split[3]:int(split[2])})
   boxes_img_rad[idx][0].append({split[3]:split[4:]})
   #print(i)
   i += 1

#Checking that labels_img_rad and boxes_img_rad are correct#    
print(labels_img_rad[:10])
print(boxes_img_rad[:10])

#This loop uses label_img_rad to fill final_labels with the sorted labels#
i = 0
for img in labels_img_rad:
   rad1 = []
   rad1f = []
   rad2 = []
   rad2f = []
   rad3 = []
   rad3f = []

   for dic in img[0]:

      if dic.keys() in rad1 or rad1 == []:
         rad1.append(dic.keys())
         rad1f.append(list(dic.values())[0])

      elif dic.keys() in rad2 or rad2 == []:
         rad2.append(dic.keys())
         rad2f.append(list(dic.values())[0])

      elif dic.keys() in rad3 or rad3 == []:
         rad3.append(dic.keys())
         rad3f.append(list(dic.values())[0])

   final_labels[i][0] = rad1f
   final_labels[i][1] = rad2f
   final_labels[i][2] = rad3f
   i += 1

    
#This loop uses boxes_img_rad to fill final_boxes with the sorted bounding boxes#
i = 0
for img in boxes_img_rad:
   rad1 = []
   rad1f = []
   rad2 = []
   rad2f = []
   rad3 = []
   rad3f = []
   for dic in img[0]:
      if dic.keys() in rad1 or rad1 == []:
         rad1.append(dic.keys())
         rad1f.append(list(dic.values())[0])
      elif dic.keys() in rad2 or rad2 == []:
         rad2.append(dic.keys())
         rad2f.append(list(dic.values())[0])
      elif dic.keys() in rad3 or rad3 == []:
         rad3.append(dic.keys())
         rad3f.append(list(dic.values())[0])

   final_boxes[i][0] = rad1f
   final_boxes[i][1] = rad2f
   final_boxes[i][2] = rad3f


   i += 1
print('first 5 elements of final boxes: ')
print(final_boxes[:5])
print('first 5 elements of final labels: ')      
print(final_labels[:5])


#Finally use the pickle module to save output#
pickle.dump( final_boxes, open( "./final_boxes.pickle", "wb" ) )
pickle.dump( final_labels, open( "./final_labels.pickle", "wb" ) )
pickle.dump( image_ids, open( "./image_ids.pickle", "wb" ) )


In [ ]:
#Import statements#
import pandas as pd
import numpy as np
import os
import pickle


First, we read in the labels from the text file in csv format

In [ ]:
import pandas as pd
import numpy as np
import os
import pickle



train_df = pd.read_csv("../input/vinbigdata-chest-xray-abnormalities-detection/train.csv")

print(train_df.head())


Initialize a list for the unique IDs in all the findings using the index method to return the number of rows in the label file, in other words it is the total number of findings in all of the images from all Radiologists.

In [ ]:

image_ids = []

f = train_df.index

print(len(f))


Lets take a look at the total number of findings, we know they eventually go with the 15000 unique images

In [ ]:
print(len(f))

To group by image, we first need a list of the unique images names from the findings.
We can do this easily using the .unique() method on the image_id column of the labels file.

In [ ]:
image_ids = list(train_df['image_id'].unique())


Just looking at a few image ids to make sure the list were populated.

In [ ]:
print(findings[59000])

print(image_ids[12])

We also need a list of image IDs for each finding

In [ ]:
findings = list(train_df['image_id'])

Initializing lists labels_img_rad will be used to create final_labels and boxes_img_rad will do the same for the associated final_boxes.

In [ ]:
labels_img_rad = []
boxes_img_rad = []
final_labels = []
final_boxes = []

The structure for final_labels and final_boxes is a list for each image containing three lists for the
findings of the three radiologists. In final_labels each list is number corresponding to the category of
that radiologist's findings, while in final_boxes each list is a list of 4 numbers corresponding to
the bounding box for that radiologist's findings.

In [ ]:
for img in image_ids:
   labels_img_rad.append([[],img])
   boxes_img_rad.append([[],img])
   final_labels.append([[],[],[]])
   final_boxes.append([[],[],[]])

# **This is where the code stops working in the notebook, as you will see final_labels and final_boxes end up as empty lists just like they started... The program works as a single cell, and final_boxes/final_labels have the correct info. It seems like a variable is getting lost or reset or something inbetween cells, if you have any ideas why this might be happening please leave a comment, I am new to kaggle environment and would appreciate any help! Thank you!**

This loop uses label_img_rad to fill final_labels with the sorted labels.

In [ ]:
i = 0
for img in labels_img_rad:
    rads = []
    rad1 = []
    rad1f = []
    rad2 = []
    rad2f = []
    rad3 = []
    rad3f = []

    for dic in img[0]:
 
       if dic.keys() in rad1 or rad1 == []:
          rad1.append(dic.keys())
          rad1f.append(list(dic.values())[0])

       elif dic.keys() in rad2 or rad2 == []:
          rad2.append(dic.keys())
          rad2f.append(list(dic.values())[0])

       elif dic.keys() in rad3 or rad3 == []:
          rad3.append(dic.keys())
          rad3f.append(list(dic.values())[0])

    final_labels[i][0] = rad1f
    final_labels[i][1] = rad2f
    final_labels[i][2] = rad3f
    i += 1
print(final_labels[:10])

This loop uses boxes_img_rad to fill final_boxes with the sorted bounding boxes.

In [ ]:
i = 0
for img in boxes_img_rad:
   rad1 = []
   rad1f = []
   rad2 = []
   rad2f = []
   rad3 = []
   rad3f = []
   for dic in img[0]:
      if dic.keys() in rad1 or rad1 == []:
         rad1.append(dic.keys())
         rad1f.append(list(dic.values())[0])
      elif dic.keys() in rad2 or rad2 == []:
         rad2.append(dic.keys())
         rad2f.append(list(dic.values())[0])
      elif dic.keys() in rad3 or rad3 == []:
         rad3.append(dic.keys())
         rad3f.append(list(dic.values())[0])

   final_boxes[i][0] = rad1f
   final_boxes[i][1] = rad2f
   final_boxes[i][2] = rad3f


   i += 1

Run the final cell to see what final_labels and final_boxes look like.

In [ ]:
print(final_boxes[:10])